In [12]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout
from matplotlib.ticker import MaxNLocator

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_Regional.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    

### Table of Contents
[1. Transportation Sector Overview](#transportation)
<br>
&ensp;[1.1 Light duty vehicles](#light-duty-vehicles)
<br>
&ensp;[1.2 Heavy duty vehicles](#heavy-duty-vehicles)
<br>
&ensp;[1.3 Buses](#buses)
<br>
&ensp;[1.4 Marine shipping](#marine)
<br>
&emsp;[1.4.1 Freight Vessel Efficiency Calculations](#freight_vessel_efficiency)
<br>
&emsp;[1.4.2 Passenger Vessel Efficiency Calculations](#pass_vessel_efficiency)
<br>
&emsp;[1.4.3 Shipping Emission Factor Calculations](#shipping_emissions)
<br>
&emsp;[1.4.4 Freight Vessel Costs](#freight_vessel_costs)
<br>
&emsp;[1.4.5 Passenger Vessel Costs](#passenger_vessel_costs)
<br>
&ensp;[1.5 Rail](#rail)
<br>
&ensp;[1.6 Aviation](#aviation)
<br>
&ensp;[1.7 Technology/commodity description lookup tool](#description_look_up)
<br>
&ensp;[1.8 Network diagram lookup tool](#network_look_up)
<br>
&ensp;[1.9 Technology/commodity look-up tool](#lookup_tool)
<br>
&ensp;[1.10 Demand Technology Specification](#demand_technologies)
<br>
&emsp;[1.10.1 Investment costs](#cost_invest)
<br>
&emsp;[1.10.2 Fixed costs](#cost_fixed)
<br>
&emsp;[1.10.3 Variable costs](#cost_variable)
<br>
&emsp;[1.10.4 Efficiency](#efficiency)
<br>
&emsp;[1.10.5 Existing Capacity](#exist_cap)
<br>
&emsp;[1.10.6 Discount Rate](#discount_rate)
<br>
&ensp;[1.11 Transportation demand](#demand)


### 1. Transportation Sector Overview <a class="anchor" id="transportation"></a>

The transportation sector in the OEO database can be split into three categories: light-duty, medium-duty, and heavy-duty vehicles. All technology names start with T_LDV, T_MDV, or T_HDV, respectively. Demand commodity names follow the format TMD(L,M,H)DV_. For example, small SUV (SS) demand is represented as TMDLDV_SS, where TMD stands for transportation miles demanded.

We represent seven light-duty vehicle size classes, outlined in [Section 1.1](#light-duty-vehicles). We externally constrain the percent of end-use demand met by each size class. As Temoa optimizes for the least-cost solution, without this constraint, the model would deploy only cheap, fuel efficient minicars and compact cars. We base size-class demand distributions on the MARKAL USEPA9r database [(Shay et al., 2013)](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883).

Medium-duty vehicles consist only of medium-duty trucks, corresponding to class 6 and 7 trucks.

Heavy-duty vehicles include heavy trucks, buses, passenger and freight rail, aviation, marine vessels, and off-highway vehicles. Each transportation technology meets a separate, exogenously specified end-use demand.  Depending on the technology, the demand may be in ton-miles, passenger-miles, or vehicle-miles traveled. Temoa does not currently allow for modal switching. For example, a long-haul truck cannot meet the same freight demand as a train; rather, the two have distinct exogenously defined demand profiles. Existing work shows that modal switching is one effective means of reducing transportation energy consumption, and we hope to incorporate modal models into the Temoa framework in the future [(Kaack et al, 2018)](https://iopscience.iop.org/article/10.1088/1748-9326/aad56c/meta).

The modeled transport modes are light-duty vehicles, trucks, buses, rail passenger, rail freight, subway, aviation, marine and off-highway. The demands for these transport modes are largely drawn from the USEPA9r database [(Shay et al., 2013)](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883) out to 2050 and can be met by various technologies. The majority of the transportation sector technologies and commodities match the most recent USEPA9r database workbooks, which as of July 2020 are VT_EPAUS9rT_TRNHDV_v20.1.0 and VT_EPAUS9rT_TRNLDV_v20.1.0 for heavy-duty and light-duty vehicles, respectively. Heavy-duty vehicle naming conventions are detailed below.

We define technology-specific discount rates, or hurdle rates, for some technologies. Hurdle rates allow the modeler to account for consumer preferences otherwise ignored in a least-cost optimization model [2]. In the light-duty vehicle sector, we assume that conventional and hybrid gasoline-fueled vehicles (excluding plug-in hybrid vehicles) have a 5% hurdle rate, equal to the model's global discount rate, and all other powertrains have a 7.5% discount rate. For heavy-duty vehicles, conventional technologies have a 5% discount rate and alternative-fueled vehicles have a 10% discount rate.

#### 1.1 Light Duty Vehicles <a class="anchor" id="light-duty-vehicles"></a>

Light-duty vehicles have the most detailed representation. Seven different size classes are specified for light-duty vehicles and each size class is constrained to contribute a fraction of total light-duty demand. We optimize the technologies and fuels within each size class. The size classes are mini compact, compact, full, small SUV, large SUV, minivan and pickup and their share in meeting light duty demand is 2.3%, 28%, 36%, 11.5%, 10%, 3.8% and 8.4%. We assume these shares remain the same throughout time. Table 1 outlines the combinations of vehicle size classes and fuels that are represented in the current database.

**Table 1.** Light-duty vehicle size class and fuel combinations represented in the database.

|                                        | Minicar | Compact | Full | Small SUV | Large SUV | Van | Pickup |
|----------------------------------------|---------|---------|------|-----------|-----------|-----|--------|
| CNG                                    |         | X       | X    |           |           | X   | X      |
| Diesel Hybrid                          |         | X       | X    | X         | X         | X   |        |
| Diesel                                 |         | X       | X    | X         | X         | X   | X      |
| E10                                    | X       | X       | X    | X         | X         | X   | X      |
| E10 Hybrid                             |         | X       | X    | X         | X         | X   | X      |
| Plug-in Hybrid: 10-mile electric range |         | X       | X    | X         |           | X   |        |
| Plug-in Hybrid: 40-mile electric range |         | X       | X    | X         |           | X   | X      |
| E85X                                   |         | X       | X    | X         | X         | X   | X      |
| EV: 100-mile range                     | X       | X       | X    |           |           | X   |        |
| EV: 200-mile range                     | X       | X       | X    | X         | X         | X   |        |
| EV: 300-mile range                     | X       | X       | X    | X         | X         |     |        |
| H2 Fuel Cell                           |         | X       | X    | X         |           | X   |        |

#### 1.2 Heavy-Duty Vehicles <a class="anchor" id="heavy-duty-vehicles"></a>

Truck technologies include heavy-duty short-haul, heavy-duty long-haul, medium-duty, and commercial.  In previous versions of the EPA database, heavy-duty trucking was split into short-haul and long-haul. This convention changed with v20.1.0; now the database only represents heavy-duty trucks in aggregate. The current database retains a distinction between short- and long-haul heavy trucks, as they differ in decarbonization technology availability. As such, the techno-economic parameters are taken from an older version of the database (v18.1.4). This older version did not include electric trucks. The characterization of electric short-haul heavy-duty trucks (T_HDV_THSELC_N) is based on technoeconomic parameters from [NREL’s electrification futures study (EFS)](https://www.nrel.gov/docs/fy18osti/70485.pdf). The parameters match those for NREL’s “Heavy Duty BEV short-range” and are an average of the medium-, slow-, and rapid-advancement scenarios. Heavy-duty short-haul trucks are assumed to travel 27,390 miles per year, in accordance with EPA’s assumptions for annual vehicle miles traveled (vmt). The existing capacity is based on EPA’s database with the addition of B20 (electric heavy-duty trucks start in 2020, so have no existing capacity). The National Biodiesel Board reports that biodiesel “ranks first among fleets for alternative fuel use,” but no explicit percentage of fleet fuel consumption is given. It is assumed that 5% of existing diesel short-haul trucks use B20 as a first-pass estimate based on a [2018 report in the Biodiesel Magazine](http://www.biodieselmagazine.com/articles/2516343/biodiesel-ranks-first-among-fleets-for-alternative-fuel-use#:~:text=North%20America's%20top%20fleets%20have,blends%20in%20their%20diesel%20vehicles).

Heavy-duty long-haul truck cost and efficiency parameters are taken from the International Council on Clean Transportation and include characterization of hydrogen fuel cell and LNG long-haul trucks. We assume that 1% of existing diesel vehicles are hybrids and an additional 5% of existing diesel vehicles run on B20. Additional data is needed to determine a more precise estimate. Similar to heavy-duty short-haul trucks, commercial truck efficiencies match v18.1.4 of the EPA databases save for electric trucks, which are drawn from NREL’s EFS. In this case, commercial trucks have the same technoeconomic parameters as NREL’s 300-mile range light-duty BEV truck [(NREL EFS)](https://www.nrel.gov/docs/fy18osti/70485.pdf). Medium-duty vehicles are consistent with USEPA9r. Table 2 lists the technologies currently represented for each truck type.

**Table 2.** Truck technologies

| Truck Technology      | Fuel / Engine Types Represented                                                                    |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Heavy Duty Short Haul | E10, E10 hybrid, diesel, diesel hybrid, B20, B20 hybrid, CNG, LPG, electricity                     |
| Heavy Duty Long Haul  | diesel, diesel hybrid, B20, B20 hybrid, LPG, LNG, hydrogen                                         |
| Commercial            | E10, E10 hybrid, diesel, diesel hybrid, E85, hydrogen fuel cell, LPG, B20, B20 hybrid, electricity |
| Medium Duty           | E10, E10 hybrid, diesel, diesel hybrid, B20, B20 hybrid, CNG, LPG, electricity                     |

#### 1.3 Buses <a class="anchor" id="buses"></a>
In accordance with the latest EPAUS9r database, we distinguish between transport buses and school buses. Demands for each bus type are specified separately. Table 3 lists the types of buses currently represented in the OEO database.

**Table 3.** Bus technologies

| Bus Type | Fuels / Engine Types Represented |
|----------|----------------------------------|
| Transit  | Diesel, diesel hybrid, E10, electricity, B20, CNG, hydrogen |
| School   | Diesel, diesel hybrid, E10, electricity, B20, LPG, CNG      |


#### 1.4 Marine  shipping <a class="anchor" id="marine"></a>
When modeling marine options, it is important to distinguish important characteristics of particular vessel types, such as annual ton-miles (or passenger-miles) traveled and engine power, and also to ensure that decarbonization options are adequately represented. For example, [(Minnehan and Pratt, 2017)](https://energy.sandia.gov/wp-content/uploads/2017/12/SAND2017-12665.pdf) note that passenger ferries may be able to operate on a battery, gaseous hydrogen, liquid hydrogen, and methanol, while a large containership’s decarbonization options are more limited. The heavy-duty shipping sector includes six vessel types: container ships, liquid bulk ships, general cargo ships, tugboats, offshore supply vessels, and passenger ships, which are given as a single category. These classifications correspond to those given in the annual publication of Waterborne Transportation Lines of the United States by the Institute for Water Resources (IWR), which provides data on the number of vessels, horsepower, and cargo or passenger capacity of U.S. flagged vessels [(U.S. Army Corps of Engineers Institute for Water Resources)](https://usace.contentdm.oclc.org/digital/collection/p16021coll2/id/3796/). The current OEO database uses only U.S. flagged vessels, and demand is only for domestic cargo and passenger transport. 

Most of the technical parameters for each vessel type were taken from the [European Environment Agency’s (EEA’s) Guidebook for International Maritime and Inland Navigation, National Navigation, National Fishing, and Recreational Boats](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019). The EEA provides emission factors (EFs), average engine power ratings, average fuel consumption, and average load factor for multiple ship types. While EEA gives technical parameters for passenger vessels, their characterization includes large international cruise ships, which skews the estimate significantly. The IWR provides data on the main engine power rating for vessels in the domestic fleet; this data is used to determine the main engine power for domestic passenger vessels. The EEA estimates are used for freight vessels, as the process of pulling vessel engine characteristics from the IWR PDF document for thousands of domestic vessels is time-consuming. Excel spreadsheets exist, but to our knowledge, the workbooks do not include vessel classification (e.g., offshore supply vessel, cargo), meaning individual vessels would have to be matched with their classification from the PDF report. This effort will be considered for future work. The IWR does not provide an estimate of auxiliary engine power, but the EEA estimates the ratio of auxiliary engine power to main engine power by ship type. This estimate is used to determine an approximate auxiliary engine power for passenger vessels.

The total existing cargo capacity of the U.S. flagged fleet (in billion ton miles) is estimated using the [Bureau of Transportation Statistics](https://www.bts.gov/product/national-transportation-statistics) annual estimates of the number of ton-miles travelled by domestic freight vessels in the United States. The historical demand is in good agreement with the domestic shipping ton-miles reported by the Energy Information Administration in the [Annual Energy Outlook 2020](https://www.eia.gov/outlooks/aeo/), which is subsequently used to forecast demand to 2050. For freight shipping, ton-miles traveled per vessel per year are estimated using an EPA workbook that pulls data from Lloyd’s register Fairplay as well as several literature studies [R. M. Weyl, R. Billings, and C. Gage, “MARKAL-MV ERG Final.” EPA Repository]. The workbook provides estimates of ton-miles traveled for containerships, general cargo ships, tankers (assumed to correspond to liquid bulk ships), tug boats, and “miscellaneous,” which is used to estimate freight work for offshore supply vessels. Annual passenger-miles traveled per vessel per year are estimated in the following way. The American Public Transportation Associations’ Public Transportation Fact Book (PTFB) reports passenger miles traveled (pmt) and vehicle miles traveled (vmt) by all ferries annually. The IWR gives the number of ferries, thus enabling the pmt per vessel per year for ferries to be calculated. The IWR “passenger” vessel category includes passenger carriers and excursion vessels in addition to ferries. To estimate the total pmt by all passenger vessels, we find the total passenger capacity for each vessel type from the IWR. We assume that the percent of the total passenger capacity by vessel type is equal to the percent of passenger miles traveled by each type. As neither vehicle miles traveled nor passenger miles traveled by each passenger vessel type are reported, passenger capacity is the only available metric to estimate the percentage of pmt demand met by each vessel type. According to the IWR, in 2017, ferries had 60% of the total passenger vessel passenger capacity. The Public Transportation Fact Book reports that ferries traveled 542.3 million passenger miles. Total demand in 2017 is then calculated as 542.3/0.6 = 908.83 million passenger miles. The EIA does not forecast passenger water transport demand for non-recreational boats; as such, demand is assumed to remain flat across the time horizon. It should be noted that passenger water transport results are less certain than freight due to the numerous estimations necessary for determining relevant parameters, but passenger transport by water represents a small fraction of total passenger transportation demand (less than 1%) so the impact on model solutions is low.

##### 1.4.1 Freight Vessel Efficiency Calculations <a class="anchor" id="freight_vessel_efficiency"></a>
Efficiency for freight water transport is found using equation 1.

Eff =  (D/(sfc x e x EC))                                                   (1)

Where 
D: freight work (billion ton- or passenger- miles traveled annually)
e: gravimetric energy density of the fuel (kWh/kg fuel)
sfc: specific fuel consumption (kg fuel/ kWh)
EC: annual energy consumption (PJ)

The gravimetric energy densities for MGO, RFO, and liquid hydrogen are given in Table 3.


**Table 3.** Conventional marine fuels and their energy densities

|     Fuel                 |     Energy Density   (kWh/kg)    |     Source  |
|--------------------------|----------------------------------|-------------|
|     Marine gasoil        |     11.84                        |     [Raucci, 2017](https://discovery.ucl.ac.uk/id/eprint/1539941/)    |
|     Residual fuel oil    |     11.25                        |     [Raucci, 2017](https://discovery.ucl.ac.uk/id/eprint/1539941/)    |
|     Liquid hydrogen      |     33.33                        |     [Raucci, 2017](https://discovery.ucl.ac.uk/id/eprint/1539941/)    |


The specific fuel consumption values for marine gasoil- and residual fuel oil-fueled vessels come from the European Environment Agency’s guidebook. The guidebook reports sfc values by engine type (high-speed diesel, medium-speed diesel, slow-speed diesel, and gas turbine). Additionally, the guidebook reports the percent of each vessel type equipped with each engine type. This data is given in Table 4. Specific fuel consumption by engine type is reported in Table 5.

**Table 4.** Vessel engine types taken from the [European Environment Agency](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019)

|                                |     Marine Gasoil    |              |              |              |     Residual Fuel Oil    |              |              |              |
|--------------------------------|----------------------|--------------|--------------|--------------|--------------------------|--------------|--------------|--------------|
|     Ship category              |     SSD              |     MSD      |     HSD      |     GT       |     SSD                  |     MSD      |     HSD      |     GT       |
|     Liquid bulk ship           |     0.191            |     0.695    |     0.114    |     0.000    |     0.776                |     0.214    |     0.008    |     0.001    |
|     Container                  |     0.898            |     0.080    |     0.022    |     0.000    |     0.943                |     0.056    |     0.001    |     0.000    |
|     General Cargo              |     0.027            |     0.645    |     0.327    |     0.000    |     0.513                |     0.480    |     0.005    |     0.001    |
|     Offshore Supply Vessel*    |     0.010            |     0.628    |     0.354    |     0.008    |     0.569                |     0.371    |     0.056    |     0.004    |
|     Tug                        |     0.000            |     0.430    |     0.567    |     0.003    |     0.000                |     0.887    |     0.113    |     0.000    |
*Offshore supply vessels are not explicitly represented; qualities for vessel category “other” are used to represent these vessels*

**Table 5.** Specific fuel consumption by engine type taken from the [European Environment Agency](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019)

|     Engine   type    |     Fuel   Type    |     SFC   [g/kWh]    |
|----------------------|--------------------|----------------------|
|     HSD              |     RFO            |     213              |
|                      |     MGO            |     203              |
|     MSD              |     RFO            |     213              |
|                      |     MGO            |     203              |
|     SSD              |     RFO            |     195              |
|                      |     MGO            |     185              |
|     GT               |     RFO            |     305              |
|                      |     MGO            |     290              |

Specific fuel consumption for each vessel type is found by multiplying the sfc for each engine type by the percent of vessels with that engine type and summing over all combinations of engine and fuel. The annual energy consumption is found by multiplying engine power by the load factor and the number of operating hours. We account for the two primary operating modes of ocean-going ships: at berth and cruising. Hours of operation for liquid bulk ships, container ships, and general cargo ships type are estimated from an International Council on Clean Transportation (ICCT) report. Operational hours represent the global fleet; these values may differ slightly for the US domestic fleet. For offshore supply vessels and tugboats, we use work by [Lindstad et al., 2017](https://www.sciencedirect.com/science/article/pii/S1361920916302723?via%3Dihub) and [Jayaram et al., 2010](https://ww2.arb.ca.gov/sites/default/files/2020-12/hybridreport1010_remediated.pdf), respectively, to estimate time spent in each operating phase.   Table 5 summarizes the time spent in each operating phase per year for each vessel type. Average engine power for each vessel type are taken from EEA and reported in Table 7.

**Table 6.** Hours spent in each operational phase by vessel type

|     Vessel Type               |     Hours spent cruising/at berth    |     Source    |
|-------------------------------|--------------------------------------|---------------|
|     Liquid Bulk               |     3313 / 4963                      |     [Comer et al., 2015](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015)      |
|     Container                 |     4940 / 3578                      |     [Comer et al., 2015](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015)      |
|     General Cargo             |     2985 / 5083                      |     [Comer et al., 2015](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015)      |
|     Offshore Supply Vessel    |     3490 / 5720                      |     [Lindstad et al., 2017](https://www.sciencedirect.com/science/article/pii/S1361920916302723?via%3Dihub)      |
|     Tug                       |     2239 / 5431                      |     [Jayaram et al., 2010](https://ww2.arb.ca.gov/sites/default/files/2020-12/hybridreport1010_remediated.pdf)      |


**Table 7.** Engine power ratings, drawn from the European Environment Agency

|     Vessel Type                |     Engine Power (kW)    |
|--------------------------------|--------------------------|
|     Liquid Bulk                |     6695                 |
|     Container                  |     22929                |
|     General Cargo              |     2657                 |
|     Tug                        |     2059                 |
|     Offshore Supply Vessel*    |     2778                 |

\**Offshore supply vessels are not explicitly represented; qualities for vessel category “other” in the EEA guidelines are used to represent these vessels*

The load factor (LF) varies depending on operational phase. According to the [EEA guidebook](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019), when cruising, the LF is 0.8 and at berth it is 0.2 (except for liquid bulk ships, for which it is 0.1). We also assume that the efficiency of new ships increases as time passes. The [Third IMO Greenhouse Gas Study](https://www.imo.org/en/OurWork/Environment/Pages/Greenhouse-Gas-Studies-2014.aspx) defines two efficiency improvement scenarios. The first assumes a 60% efficiency improvement in 2050 relative to 2012 levels and the second assumes a 40% efficiency improvement. We conservatively assume the latter, implementing it as a 5% efficiency improvement in every 5-year time period starting in 2020.

We base hydrogen vessel feasibility estimates on the Sandia report [(Minnehan and Pratt, 2017)](https://energy.sandia.gov/wp-content/uploads/2017/12/SAND2017-12665.pdf), which find that the majority of vessel classes are capable of running on liquefied hydrogen. Of the thirteen existing vessels considered in their analysis, they found ten would be able to run on liquid hydrogen based on voyage distance and mass and volume limitations. These vessels included a container ship, a general cargo ship, an offshore supply vessel, and a bulk cargo ship. Therefore, we define all freight shipping technologies with a liquid hydrogen option except tugboats, which were not considered in the study. Furthermore, a [2020 ICCT report](https://theicct.org/publications/zero-emission-container-corridor-hydrogen-2020) found that 99% of voyages made along the international shipping corridor between the San Pedro Bay and the Pearl River Delta could be fueled by hydrogen. We assume hydrogen ships will be available by 2030, based on an estimate from Flexport, according to an [NPR report](https://www.npr.org/2019/07/16/716693006/the-dawn-of-low-carbon-shipping#:~:text=via%20Getty%20Images-,Container%20ships%20and%20other%20maritime%20vessels%20currently%20run%20on%20pollutant,new%20fuels%20such%20as%20hydrogen). This is likely a conservative estimate, as some European countries plan to deploy hydrogen ships as early as 2021 ([see this article](https://www.offshore-energy.biz/flagships-project-gets-usd-5-6-mn-for-two-hydrogen-fueled-ships/#:~:text=Flagships%20Project%20Gets%20USD%205.6%20Mn%20for%20Two%20Hydrogen%2DFueled%20Ships,-Business%20%26%20Finance&text=The%20European%20innovation%20project%20Flagships,vessels%20in%20France%20and%20Norway.)).  To calculate vessel efficiencies, we use a slightly simplified version of the method outlined above. We use a single value for specific fuel consumption, taken from [Raucci, 2017](https://discovery.ucl.ac.uk/id/eprint/1539941/), rather than a weighted average. Otherwise, the calculation methodology is the same.

##### 1.4.2 Passenger Vessel Efficiency Calculations <a class="anchor" id="pass_vessel_efficiency"></a>
Efficiency calculations for passenger vessels deviate from the EEA methodology as the EEA report includes large ocean-going cruise ships in their passenger vessel classification. The IWR reports only one such vessel in the US merchant fleet, so EEA values are not appropriate. Passenger vessel efficiency is based on [(Cottrell, 2011)](https://www.mdpi.com/1996-1073/4/2/239), whose work compared US ferry boat efficiencies to highway bridges crossing the same body of water. Cottrell’s work gives passenger-miles per gallon for eight ferry vessel companies operating 96 marine gasoil-powered ferries. Using this data yields an average efficiency of 0.092 billion passenger miles traveled per PJ. To benchmark both passenger and freight vessel efficiencies, as well as demand estimates, we compare total energy consumption for domestic shipping from our calculations to Annual Energy Outlook forecasts.

We don’t consider a hydrogen-powered passenger vessel, but model electric passenger vessels based on the E-Ferry, an electric ferry currently operating in Southern Denmark. Based on energy consumption data from an EU-sponsored evaluation report on the E-Ferry, efficiency is found to be 0.165 billion passenger miles per PJ.

##### 1.4.3 Shipping Emission Factor Calculations <a class="anchor" id="shipping_emissions"></a>
Emission factors (kt/ billion ton- or passenger-miles traveled) are calculated using the EEA Tier II methodology. Similarly to efficiency, emission factors are reported by engine type.  The emission factors for each vessel type are calculated using the percent of each vessel type using a particular engine and are reported in Table 8.

**Table 8.** Emission Factors, kg/tonne fuel [European Environment Agency](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019)

|                         |     Fuel     |     NOx EF, 2000     |     NOx EF, 2005     |     NOx EF, 2010     |     PM10     |     PM2.5     |     NMVOC     |
|-------------------------|--------------|----------------------|----------------------|----------------------|--------------|---------------|---------------|
|     Liquid Bulk         |     RFO      |     86.62            |     83.84            |     80.75            |     7.60     |     6.81      |     2.7       |
|                         |     MGO      |     69.46            |     67.28            |     64.91            |     1.52     |     1.34      |     2.8       |
|     General Cargo       |     RFO      |     79.53            |     76.91            |     74.16            |     6.31     |     5.66      |     2.7       |
|                         |     MGO      |     63.81            |     61.84            |     59.51            |     1.50     |     1.31      |     2.8       |
|     Container           |     RFO      |     91.24            |     88.35            |     85.05            |     8.42     |     7.55      |     2.7       |
|                         |     MGO      |     89.02            |     85.86            |     83.73            |     1.59     |     1.48      |     2.8       |
|     Tug                 |     RFO      |     65.01            |     62.76            |     60.66            |     3.80     |     3.40      |     2.7       |
|                         |     MGO      |     61.52            |     59.56            |     57.35            |     1.50     |     1.30      |     2.8       |
|     Offshore Supply     |     RFO      |     80.62            |     77.99            |     75.17            |     6.58     |     5.89      |     2.7       |
|                         |     MGO      |     62.82            |     60.88            |     58.57            |     1.49     |     1.29      |     2.8       |
|     Passenger           |     RFO      |     65.01            |     62.75            |     60.65            |     3.88     |     3.48      |     2.7       |
|                         |     MGO      |     48.13            |     46.61            |     44.85            |     0.99     |     0.86      |     2.8       |

The values in Table 8 are converted from kg/tonne fuel to kt/btm or kt/bpm by multiplying the specific fuel consumption (g/kWh), the total annual energy consumption (kWh/year) and the ton- or passenger-miles traveled in a year. SO2 emissions are calculated using equation 2.

 EF_SO2 = 0.02 × %_sulfur × 0.98 × sfc                                       (2)

This equation is taken from the [Third IMO Greenhouse Gas Study](https://www.imo.org/en/OurWork/Environment/Pages/Greenhouse-Gas-Studies-2014.aspx), which includes the constant 0.98 as approximately 98% of fuel sulfur is converted to gaseous SO2. As of January 1, 2015, ships inside a sulfur emissions control area must use fuel that that is less than or equal to 0.1% m/m (or equivalently strict emissions control technology) [(Sulphur oxides (SOx) – Regulation 14)](https://www.imo.org/en/OurWork/Environment/Pages/Sulphur-oxides-(SOx)-%E2%80%93-Regulation-14.aspx). To meet sulfur requirements, ships can use residual fuel oil (RFO) with a scrubber, marine gas oil, or an alternative fuel such as liquefied natural gas (LNG) or hydrogen [(Zis et al., 2016)](https://journals.sagepub.com/doi/abs/10.3141/2549-05?journalCode=trra).  Additionally, ships must meet nitrogen oxide standards based on their construction date (see Table 9). It is assumed here that all existing vintages meet Tier II standards while new builds meet Tier III standards and that all vessels meet the sulfur standard, since it was implemented before the model’s first time period (2017). To meet NOx requirements, selective catalytic reduction (SCR) or exhaust gas recirculation (EGR) technologies can be retrofitted onto vessels or alternative fuels may be used [(Jonge et al., 2005a)](https://ec.europa.eu/environment/air/pdf/task2_nox.pdf). For simplicity, it is assumed that all vessels using MGO are equipped with SCR technology and all vessels using RFO are equipped both with a scrubber and SCR. Relevant assumptions for emission factor calculations are that scrubbers reduce SO2 by 75% and PM by 25% from the ship and engine type baseline and selective catalytic reduction (SCR) reduces NOx by 90% [(Jonge et al., 2005a)](https://ec.europa.eu/environment/air/pdf/task2_nox.pdf),[(Jonge et al., 2005b)](https://ec.europa.eu/environment/air/pdf/task2_so2.pdf).

**Table 9.** IMO NOx Emissions Standards

|     Tier    |     Ship   construction date on or after    |     <td colspan=3>**Total Weighted cycle emission limit   (g/kWh) n = engine’s rated speed (rpm)**    |                       |                  |
|-------------|---------------------------------------------|-------------------------------------------------------------------------------------|-----------------------|------------------|
|             |                                             |     n < 130                                                                         |     n = 130 – 1999    |     n >= 2000    |
|     I       |     1 January 2000                          |     17.0                                                                            |     45*n^-0.2          |     9.8          |
|     II      |     1 January 2011                          |     14.4                                                                            |     44*n^-0.23         |     7.7          |
|     III     |     1 January 2016                          |     3.4                                                                             |     9*n^-0.2           |     2.0          |


##### 1.4.4 Freight Vessel Costs <a class="anchor" id="freight_vessel_costs"></a>
Freight vessel costs are derived from [(Taljegard et al., 2014)](https://pubs.acs.org/doi/abs/10.1021/es5018575).  Costs are given for deep sea, short sea, and containerships. We assume that liquid bulk ships and general cargo ships are deep sea ships while offshore vessels and tugboats are short sea ships. Costs are reported by Taljegard et al. in USD/kW. These values are multiplied by the engine size from EEA to obtain values in USD.

Operation and maintenance costs for RFO / MGO-fueled vessels are assumed to be 5% of capital costs. We also include costs for emission control technologies (both investment and operation \& maintenance), which are taken from Entec UK Limited [(Jonge et al., 2005a)](https://ec.europa.eu/environment/air/pdf/task2_nox.pdf),[(Jonge et al., 2005b)](https://ec.europa.eu/environment/air/pdf/task2_so2.pdf). It is assumed that selective catalytic reduction technology costs \$0.5M and a scrubber costs \$6.5M. Operation and maintenance costs depend on the fuel being used. For RFO-fueled vessels, SCR O\&M costs are \$7.3/MWh. For MGO-fueled vessels, they are \$4.2/MWh. Scrubber O\&M costs are set at \$0.778/MWh. Costs are converted from 2004 Euros assuming 1.24 USD to Euro 2004 [source](https://www.macrotrends.net/2548/euro-dollar-exchange-rate-historical-chart) and 1.255 USD 2004 to USD 2015 [source](https://www.minneapolisfed.org:443/about-us/monetary-policy/inflation-calculator).

##### 1.4.5 Passenger Vessel Costs <a class="anchor" id="passenger_vessel_costs"></a>
Passenger vessel costs are based on the [E-Ferry report](https://www.xn--el-frgeprojekt-3ib.dk/nyheder/e-ferry-ellen-crosses-the-finish-line-and-delivers-great-results), which compares the novel electric ferry to existing marine gasoil ferries. A passenger vessel is assumed to cost 12,855,657 2019 EUR, which is converted to 16545.6 M USD 2015/bpm. Operation and maintenance costs are reported to be 2455 M USD 2015/bpm (converted from 1907618 EUR 2019). Emission control technologies are also added to passenger vessel costs.

#### 1.5 Rail  <a class="anchor" id="rail"></a>
Rail is split into intercity rail, subways, commuter rail (aggregated to meet total passenger rail demand) and freight rail. Subway efficiencies are drawn from the EPA’s 9-region database. Intercity and freight rail efficiencies are taken from [GREET](https://greet.es.anl.gov/) and commuter rail efficiencies are estimated based on simulations run by [Isaac at UC Davis](https://ncst.ucdavis.edu/research-product/fuels-and-fuel-technologies-powering-21st-century-passenger-and-freight-rail).

Passengers per vehicle mile traveled (load factor) is calculated as the ratio of total passenger miles traveled to total vehicle miles traveled in the US. This value is found using data from the [Public Transportation Fact Book](https://www.apta.com/research-technical-resources/transit-statistics/public-transportation-fact-book/).  Efficiency for dual-mode trains is assumed to be the average of diesel and electric efficiencies as a first pass estimate until better data is found and a more accurate assumption can be made. Table 10 outlines the fuels modeled for each category of rail technology.

**Table 10.** Fuels available for rail by category

|     Rail Category             |     Fuels represented                                                         |
|-------------------------------|-------------------------------------------------------------------------------|
|     Freight Rail              |     Diesel, B20, B100,   LNG                                                  |
|     Commuter Rail             |     Diesel, electric,   B20, dual-mode (can operate as diesel or electric)    |
|     Intercity rail            |     Diesel, B20, B100,   hydrogen                                             |
|     Subways and streetcars    |     Electricity                                                               |

Capital costs are estimated using several sources. Freight rail and intercity rail capital costs are taken from [(Isaac and Fulton, 2016)](https://steps.ucdavis.edu/wp-content/uploads/2017/10/RAPHAELISAAC-2017-UCD-ITS-RP-17-02-2.pdf). Variable costs are set to 6% of capital costs for each technology type. For commuter rail, Gattuso and Restuccia present estimates for several different commuter rail trains, each with differ passenger capacity and number of train cars [(Gattuso and Restuccia, 2014)](https://www.sciencedirect.com/science/article/pii/S1877042814000895). We assume a representative train has three cars and as such, costs are adjusted to a three-car baseline. For example, Gattuso and Restuccia report that the AnsaldoBreda RegioStar has 8 cars and costs 8.55 M USD 2015 (converted from M EUR 2013). This is converted to a three-car basis by multiplying the total train cost by 3/8. Additional data is collected from several news articles noting the purchase of electric and diesel multiple unit trains [(source, ](https://metrarail.com/about-metra/newsroom/facts-about-metras-highliner-purchase ) [source)](https://www.denverpost.com/2014/12/02/rtds-new-rail-cars-for-fastracks-system-unveiled-to-public/). Currently, the database does not contain operation and maintenance costs for commuter rail.

#### 1.6 Aviation  <a class="anchor" id="aviation"></a>
Per the USEPA9r, aviation is split into passenger jets and general aviation. General aviation is fueled by gasoline and passenger jets by jet fuel. Other than synthetic fuels, there are currently no alternative fuels for aviation characterized in the database. All efficiency and cost parameters are drawn from the USEPA9r database.

#### 1.7 Technology/commodity description lookup tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. LDV, shipping, rail). The tool provides a list of all the technologies in the database that may be relevant to the query.

In [13]:
w = widgets.Text(value='LDV')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='LDV')

Output()

#### 1.8 Network diagram lookup tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [14]:
w = widgets.Text(value='TMDHDV_WTP')
display(w)
def f(w):
    if len(w)>0:
        df = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df['comm_desc'] = df['comm_desc'].str.replace('#','').str.strip()
    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['comm_desc'][0]))
                final_dem = df['comm_name'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="12,12";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=0,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='TMDHDV_WTP')

Output()

#### 1.9 Technology/commodity look-up tool <a class="anchor" id="lookup"></a>
Use the tool below to retrieve the description for any technology or commodity within the database. Type the commodity or technology name in the box below to view the description. Note that names are case sensitive.

In [15]:
w = widgets.Text(value='ELC')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown((df['comm_desc'].values[0].replace('#', '').strip())))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='ELC')

Output()

#### 1.10 Demand Technology Specification <a class="anchor" id="demand_technologies"></a>

Techno-economic parameters for an individual technology are assumed to be the same for all regions. Currently, existing capacity has been divided equally among the nine regions, however, this will be revisited in the near-term, using the state-level representation in the MARKAL database, to aggregate to the OEO regions. Relevant techno-economic paramters can be queried and displayed in the sections that follow. Note that all costs will be updated to 2018$ in the near-term.

##### 1.10.1 Investment costs <a class="anchor" id="cost_invest"></a>

In [16]:
def show_cost_invest(con):
    
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql(
        "SELECT regions, tech,vintage, cost_invest, cost_invest_units FROM CostInvest WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport' ) ORDER BY tech, vintage",
        con)

    display_types = ['table', 'figure']
    
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()


    def filter_tech(tech='', region='', o_format=''):
        if tech == 'All':
            df_sel = df[(df.regions == region)]
        else:
            df_sel = df[(df.agg_tech.str.contains(tech)) & (df.regions == region)]
        if o_format == 'table':
            df_sel = df_sel.pivot_table(
                index=['regions', 'tech', 'agg_tech', 'cost_invest_units'],
                columns='vintage',
                values='cost_invest').reset_index().set_index('regions')
            df_sel.rename(columns={'cost_invest_units': 'units', 'agg_tech':'description'}, inplace=True)
            df_sel['units'] = df_sel['units'].str.replace('#','').str.replace('M$','$M').str.strip()
            if len(df_sel['units'].unique())==1:
                df_sel['units'] = df_sel['units'].unique()[0].replace('M$','$M')
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif (o_format == 'figure') & (tech!='All') :
            fig, ax = plt.subplots(figsize=(10, 6))
            for ind_tech in df_sel.tech.unique():
                plt.plot(df_sel[df_sel.tech == ind_tech].vintage,
                         df_sel[df_sel.tech == ind_tech].cost_invest,
                         label=ind_tech)
            plt.legend()
            plt.ylabel('Investment costs (' + df_sel['cost_invest_units'].unique()[0].replace('#','').strip().replace('M$','$M') + ')')
            #plt.ylim([0, df.cost_invest.max() * 1.1])
            plt.xlabel('Vintage')


    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_invest(con)

##### 1.10.2 Fixed costs <a class="anchor" id="cost_fixed"></a>

In [17]:
def show_cost_fixed(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech,vintage, cost_fixed_units, AVG(cost_fixed) AS cost_fixed FROM CostFixed WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport') GROUP BY regions, tech,vintage ORDER BY tech, vintage", con)
    display_types = ['table', 'figure']
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())

    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))    
    regions = [x for x in df.regions.unique() if '-' not in x]
    def filter_tech(tech ='', region = '', o_format=''):
        if tech=='All':
            df_sel = df[(df.regions==region)].copy()
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        if o_format=='table':
            #df_sel = df_sel.pivot_table(index=['regions','tech', 'cost_fixed_units'], columns='vintage', values='cost_fixed').reset_index().set_index('regions')
            df_sel = df_sel[['regions','tech','agg_tech','vintage','cost_fixed_units','cost_fixed']]
            df_sel['cost_fixed_units'] = df_sel['cost_fixed_units'].str.replace('#','').str.replace('M$','$M').str.strip()
            header = ['regions', 'technology', 'description','vintage', 'units', 'fixed cost']
            display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header,  floatfmt=".0f" , tablefmt='html')))

        elif o_format=='figure':
            fig, ax = plt.subplots(figsize=(10,6))
            for ind_tech in df_sel.tech.unique():
                plt.bar(df_sel[df_sel.tech==ind_tech].vintage, df_sel[df_sel.tech==ind_tech].cost_fixed, label=ind_tech)
            plt.legend()
            if len(df_sel)==1:
                plt.xlim([df_sel.vintage.values - 2, df_sel.vintage.values + 2])
                plt.xticks(np.arange(df_sel.vintage.values - 1, df_sel.vintage.values + 2))
            plt.ylabel('Fixed costs ($M/GWyr)')
            #plt.ylim([0, df.cost_fixed.max()*1.1])
            plt.xlabel('Vintage')

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_fixed(con)

##### 1.10.3 Variable costs <a class="anchor" id="cost_variable"></a>

In [18]:
def show_cost_variable(con):

    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql("SELECT regions, tech,vintage, cost_variable_units, AVG(cost_variable) AS cost_variable FROM CostVariable WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport') AND cost_variable> 0 GROUP BY regions, tech,vintage ORDER BY vintage", con)
    display_types = ['table', 'figure']
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())

    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech ='', region = '', o_format=''):
        if tech=='All':
            df_sel = df[(df.regions==region)].copy()
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        if o_format=='table':
            #df_sel = df_sel.pivot_table(index=['regions','tech', 'cost_variable_units'], columns='vintage', values='cost_variable').reset_index().set_index('regions')
            df_sel = df_sel[['regions','tech','agg_tech','vintage','cost_variable_units','cost_variable']]
            header = ['regions', 'technology', 'decription','vintage', 'units', 'variable cost']
            display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header,  floatfmt=".2f" , tablefmt='html')))
        elif o_format=='figure':
            fig, ax = plt.subplots(figsize=(10,6))
            for ind_tech in df_sel.tech.unique():
                plt.bar(df_sel[df_sel.tech==ind_tech].vintage, df_sel[df_sel.tech==ind_tech].cost_variable, label=ind_tech)
            plt.legend()
            if len(df_sel)==1:
                plt.xlim([df_sel.vintage.values - 2, df_sel.vintage.values + 2])
                plt.xticks(np.arange(df_sel.vintage.values - 1, df_sel.vintage.values + 2))
            plt.ylabel('Variable costs ($M/PJ)')
            #plt.ylim([0, df.cost_variable.max()*1.1])
            plt.xlabel('Vintage')
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)

show_cost_variable(con)

##### 1.10.4 Efficiency <a class="anchor" id="efficiency"></a>

In [19]:
def show_efficiency(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, efficiency FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df#df.drop("vintage", axis=1).groupby(by = ['regions','agg_tech']).sum().reset_index()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
            df_sel.efficiency = 100*df_sel.efficiency
        df_sel = df_sel[['regions','tech','agg_tech','vintage','efficiency']]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description","vintage",'efficiency (%)' ], floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_efficiency(con)

##### 1.10.5 Existing Capacity <a class="anchor" id="exist_cap"></a>

In [20]:
def show_exist_cap(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech,vintage, exist_cap, exist_cap_units FROM ExistingCapacity WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df.drop("vintage", axis=1).groupby(by = ['regions','tech','agg_tech','exist_cap_units']).sum().reset_index()
    df_sum.sort_values(by='exist_cap', ascending=False, inplace=True)
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description", "units","capacity" ], floatfmt=".1f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_exist_cap(con)


##### 1.10.6 Discount Rate <a class="anchor" id="discount_rate"></a>

In [21]:
def show_disc_rate(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, tech_rate FROM DiscountRate \
    WHERE tech in (SELECT tech FROM technologies WHERE sector='transport')", con)
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        if tech=='All':
            df_sel = df[(df.regions==region)]
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        df_sel = df_sel.pivot_table(index=['regions','tech', 'agg_tech'], columns='vintage', values='tech_rate').reset_index()
        df_sel.rename(columns={'agg_tech':'description'},inplace=True)
        header = list(df_sel.columns.values)
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header, floatfmt=".3f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_disc_rate(con)

#### 1.11 Transportation demand <a class="anchor" id="demand"></a>
The <a href="https://www.nrel.gov/docs/fy18osti/71500.pdf" rel="nofollow">Electrification Futures Study (EFS) Scenarios of Electric Technology Adoption and Power Consumption for the United States</a> report was published in 2018 and presents a range of demand scenarios for all the energy sectors in the United States. Specifically, the study estimates transportation-related service demands, as shown in Table 11, primarily by downscaling service demands data from the US EIA 2017 Annual Energy Outlook and the NEMS model.

EIA service demand data were made available by US Census Division to the EFS study. These Census Division-level data were then downscaled to the state-level in the EFS by using factors relevant to each end-use demand. These state-level demand projectons to 2050 were made available by the [EFS](https://data.nrel.gov/submissions/92), for the demand types listed in Table 11.

These state-level demands are aggregated, by demand category, to the nine OEO nine-regional level using the mapping shown in Figure 1 of the Overview notebook. Using these aggregated data, the fraction of demand by category in each OEO region are estimated for every year. For example, light duty vehicle transportation demand in the Northeast region is around 9% of US total demand in 2050.

The national level demands in the US_National database, which were developing by aggregating data from the US EPA MARKAL database [(Lenox, et al., 2013)](https://cfpub.epa.gov/si/si_public_record_report.cfm?Lab=NRMRL&dirEntryId=278925),
 were then scaled to each of the nine OEO regions using the regional demand fractions estimated using the EFS. The demand categories in the EFS were mapped to the OEO categories, as shown in Table 11.
 
 **Table 11.** Annual service demands categories in the EFS, mapped to the OEO demands

| Demand category | Unit | OEO demands |
|-|-|-|
| Aviation <img width=200/>| SEAT_MILE | transport passenger miles demanded for HDV airplane|
| Freight rail | GIGATON_MILE | transport miles demanded for HDV rail freight |
| Heavy duty trucks | GIGAMILE | transport miles demanded for HDV heavy truck long haul, transport miles   demanded for HDV heavy truck short haul |
| Light duty autos | MILE | transport miles demanded for LDV, transport miles demanded for offhighway   diesel, transport miles demanded for offhighway E10, demand for freight   shipping, demand for passenger ships |
| Light duty trucks | MILE | transport miles demanded for HDV commercial truck |
| Medium duty trucks | GIGAMILE | transport miles demanded for medium duty truck |
| Passenger rail | PASSENGER_MILE | transport miles demanded for HDV rail passenger |
| School and intercity buses | PASSENGER_MILE | transport miles demanded for school bus |
| Transit buses | MILE | transport miles demanded for transit bus |


End-use regional OEO demand projections can be explored in the tool below.

In [22]:
def show_demand(con):

    display_types = ['table', 'figure']

    df_demand = pd.read_sql("SELECT * FROM Demand WHERE demand_comm in (SELECT comm_name FROM commodities WHERE flag='d') \
                            AND demand_comm in (SELECT output_comm FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector='transport'))", con)
    
    df_tech_desc = pd.read_sql("SELECT comm_name, comm_desc FROM commodities", con)
    df_demand['demand_comm'] = df_demand['demand_comm'].map(lambda x: df_tech_desc.loc[df_tech_desc.comm_name==x,'comm_desc'].values[0].replace('#','').strip())
    
    demand_comms = []
    for unique_demand_comm in df_demand.demand_comm.unique():
        demand_comms.append(unique_demand_comm)
    demand_comms = list(set(demand_comms))
    regions = df_demand.regions.unique()


    def filter_demand_comm(demand_comm='', o_format=''):
        demand_comm_name = pd.read_sql("SELECT comm_desc FROM commodities where comm_name=='" + demand_comm +"'", con)
        df_sel = df_demand[(df_demand.demand_comm.str.contains(demand_comm))]
        df_sel = df_sel.pivot_table(
            index=['regions', 'demand_comm', 'demand_units'],
            columns='periods',
            values='demand').reset_index().set_index('regions')
        #df_sel['demand_comm'] = demand_comm_name.iloc[0,0].replace('#','').strip()
        df_sel.rename(columns={'demand_units': 'units', 'demand_comm':'demand'}, inplace=True)
        df_sel.units = df_sel.units.str.replace('#','').str.strip()
        if o_format == 'table':
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif o_format == 'figure':
            fig, ax = plt.subplots(figsize=(10, 8))
            df_sel.drop(columns=['demand', 'units']).transpose().plot.bar(stacked=True, ax = ax)
            plt.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles[::-1], labels[::-1], loc='upper right',bbox_to_anchor=(1.2, 1))
            plt.ylabel(df_sel['demand'].unique()[0] + ' (' + df_sel.units.unique()[0].replace('#','') + ')')
            plt.xlabel('')

    display(Markdown('Annual end-use regional demand projections to 2050'))
    w2 = widgets.Select(options=demand_comms, description='demand')
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_demand_comm, demand_comm=w2, o_format=w3)


    controls_rows(w)

show_demand(con)

Annual end-use regional demand projections to 2050